In [1]:
from keras.models  import load_model
import keras.backend as K
from sklearn.preprocessing import normalize
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
from colors import color
import pandas as pd
import numpy as np

aspects = [str(i) for i in range(4)] #app, aroma, palate,taste

Using TensorFlow backend.


[]


In [2]:
vec = cPickle.load(open('../../beer_data/beer_vec_ds_2000.p', 'rb'))

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

ds = pd.read_csv('../../beer_data/beer_ds.csv')

from ast import literal_eval as make_tuple
ds['bits'] = ds['bits'].map(lambda s : make_tuple(s))

aspect_columns = sorted(['review/appearance', 'review/taste', 'review/aroma', 'review/palate'])
aspects = [str(i) for i in range(4)]

train_idxs, val_idxs = train_test_split(ds.index, stratify=ds['bits'], train_size=0.9, random_state=1337)

/home/sarthak/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
idxs = val_idxs
H = {}
for i in range(4) :
    H[str(i)] = np.zeros((len(idxs), len(idxs)))
    a0 = set(ds[ds[aspect_columns[i]] == 0].index) & set(idxs)
    a1 = set(ds[ds[aspect_columns[i]] == 1].index) & set(idxs)
    a0 = map(lambda s : list(idxs).index(s), a0)
    a1 = map(lambda s : list(idxs).index(s), a1)
    for j in a0 :
        H[str(i)][j, a0] = 1
    for j in a1 :
        H[str(i)][j, a1] = 1

    H[str(i)][np.arange(len(idxs)), np.arange(len(idxs))] = 0

In [6]:
model = load_model('../store/weights/beer/Gated2CNNModelSunDec1015:45:442017testingsigmoid//loss.h5', 
                   custom_objects={'contrastive_loss' : lambda a,b:b, '<lambda>' : lambda a,b : b})

In [7]:
model_c1 = {}
for aspect in aspects :
    model_c1[aspect] = K.function(model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).inputs, 
                          [model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).layers[16].output])

In [8]:
X = vec.X[val_idxs]

In [21]:
filters_pico = {k:[] for k in aspects}
X = X[-2:]
i, bs = 0, 100
while i*bs < len(X):
    for aspect in aspects :
        val = model_c1[aspect]([X[i*bs:(i+1)*bs]])[0][:, :, 0]
        val = np.dstack((X[i*bs:(i+1)*bs], val))
        filters_pico[aspect].append(val)
    i+=1
        
for aspect in aspects :
    filters_pico[aspect] = np.concatenate(filters_pico[aspect])
    print filters_pico[aspect].shape

(2, 305, 2)
(2, 305, 2)
(2, 305, 2)
(2, 305, 2)


In [56]:
s = -1
from random import random
#30, 50, 60, 95, 120, 150, 7000

nz = sum([1 for i in filters_pico['3'][s, :, 0] if i == 0])
gate0 = filters_pico['0'][s, :, 1]
gate1 = filters_pico['1'][s, :, 1]
gate2 = filters_pico['2'][s, :, 1]
gate3 = filters_pico['3'][s, :, 1]

gate0[:nz] = 1.
gate0 = np.convolve(gate0, [1./3 for _ in range(3)], mode='same')
gate1 = np.convolve(gate1, [1./3 for _ in range(3)], mode='same')
gate2 = np.convolve(gate2, [1./3 for _ in range(3)], mode='same')
gate3 = np.convolve(gate3, [1./3 for _ in range(3)], mode='same')

gate0 = gate0 - gate0[nz:].mean(axis=0)
gate1 = gate1 - gate1[nz:].mean(axis=0)
gate2 = gate2 - gate2[nz:].mean(axis=0)
gate3 = gate3 - gate3[nz:].mean(axis=0)

gate0 = gate0 / gate0.max(axis=0)
gate1 = gate1 / gate1.max(axis=0)
gate2 = gate2 / gate2.max(axis=0)
gate3 = gate3 / gate3.max(axis=0)

gate0 = gate0.clip(0, 1)
gate1 = gate1.clip(0, 1)
gate2 = gate2.clip(0, 1)
gate3 = gate3.clip(0, 1)



print gate3[nz:], gate0[nz:]
gate4 = np.vstack([gate0, gate1, gate2, gate3])
gate4 = np.argmax(gate4, axis=0)

#print gate0
X_words = [vec.idx2word[int(i)] for i in filters_pico['3'][s, :, 0]]
c = ['red', 'green', 'blue', 'yellow']
for i, j in zip(X_words, gate4.T) :
    if i != '[0]' :
        print color(i, bg=c[j]),
print ""
for i, j in zip(X_words, gate0):
    if i != '[0]':
        #if j < 0.4 : j = 0
        print color(i, bg=[int(255),int((1-j)*255), int((1-j)*255)]),
# for i, j in zip(X_words, gate0):
#     if i != '[0]':
#         print '{\setlength{\\fboxsep}{0pt}\colorbox{red!'+ str(int(j*100)) + '!white}{'+i+'}}',
print ""
for i, j in zip(X_words, gate1) :
    if i != '[0]':
        #if j < 0.4 : j = 0
        print color(i, bg=[int((1-j)*255), int(255), int((1-j)*255)]),
# for i, j in zip(X_words, gate1):
#     if i != '[0]':
#         print '{\setlength{\\fboxsep}{0pt}\colorbox{green!'+ str(int(j*100)) + '!white}{'+i+'}}',
print ""

for i, j in zip(X_words, gate2) :
    if i != '[0]':
        #if j < 0.4 : j = 0
        print color(i, bg=[int((1-j)*255), int((1-j)*255), int(255)]),
# for i, j in zip(X_words, gate2):
#     if i != '[0]':
#         print '{\setlength{\\fboxsep}{0pt}\colorbox{blue!'+ str(int(j*100)) + '!white}{'+i+'}}',
print ""
for i, j in zip(X_words, gate3) :
    if i != '[0]':
        #if j < 0.4 : j = 0
        print color(i, bg=[int(255), int((1-j)*255), int(255)]),
# for i, j in zip(X_words, gate3):
#     if i != '[0]':
#         print '{\setlength{\\fboxsep}{0pt}\colorbox{yellow!'+ str(int(j*100)) + '!white}{'+i+'}}',

[ 0.          0.          0.          0.          0.          0.35327138
  0.52125588  0.4248944   0.25071592  0.14725121  0.19651399  0.          0.
  0.          0.          0.          0.          0.03698237  0.01129183
  0.          0.          0.11531213  0.25677586  0.11324884  0.          0.
  0.          0.02476721  0.41915582  0.59964804  0.39863117  0.10912928
  0.17325796  0.31799579  0.26279002  0.47787685  0.91308683  1.
  0.94886685  0.49929893  0.65839284  0.39444359  0.13806057  0.          0.
  0.08405309  0.11133996  0.        ] [ 0.2999336   0.06353321  0.16878274  0.2526388   0.20492785  0.14846945
  0.1269808   0.25804949  0.36477396  0.36642014  0.22873796  0.20717621
  0.19644356  0.13824173  0.05178962  0.05446577  0.03670126  0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.      

In [51]:
activated_words = {}
for aspect in aspects :
    filters = filters_pico[aspect]
    word_activations = np.zeros((vec.vocab_size, 1))
    word2filter = filters.reshape(-1, filters.shape[-1])
    for word in word2filter :
        word_activations[int(word[0]), 0] = word[1]
    activated_words[aspect] = word_activations
    
joint = np.concatenate([activated_words[aspect] for aspect in aspects], axis=1)

In [56]:
top_words_df = pd.DataFrame()
top_words_size = 10
for aspect in aspects :
    popwords = joint[:, int(aspect)]
    sorted_idx = np.argsort(popwords)
    top_words = sorted_idx[-top_words_size:]
    print [str(v) for k, v in vec.idx2word.items() if k in top_words]
    top_words = pd.Series([v for k, v in vec.idx2word.items() if k in top_words])
    
    top_words_df[aspect] = top_words
top_words_df

['attractive', 'beautiful', 'fingers', 'pumpkin', 'quarter', 'received', 'retention', 'sheets', 'sipper', 'well-balanced']
['beer', 'cardboard', 'cheap', 'down', 'follows', 'medium-light', 'rice', 'settled', 'skunked', 'skunky']
['bother', 'crafted', 'luscious', 'mellow', 'mint', 'range', 'recommended', 'roasted', 'tasting', 'weight']
['amazingly', 'down', 'highly', 'product', 'recommended', 'tasted', 'thoroughly', 'to', 'truly', 'wow']


,0,1,2,3
0,attractive,beer,bother,amazingly
1,beautiful,cardboard,crafted,down
2,fingers,cheap,luscious,highly
3,pumpkin,down,mellow,product
4,quarter,follows,mint,recommended
5,received,medium-light,range,tasted
6,retention,rice,recommended,thoroughly
7,sheets,settled,roasted,to
8,sipper,skunked,tasting,truly
9,well-balanced,skunky,weight,wow


In [123]:
import keras.backend as K
model_e = {}
for aspect in aspects :
    model_e[aspect] = K.function(
                model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).inputs + [K.learning_phase()], 
                model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).outputs)
embeds = {}
vecs = {k:[] for k in aspects}
i, bs = 0, 100
while i*bs < len(val_idxs):
    for aspect in aspects :
        result = model_e[aspect]([X[i*bs:(i+1)*bs], 0])[0]
        vecs[aspect].append(result)
    i += 1

for aspect in aspects :
    embeds[aspect] = np.concatenate(vecs[aspect], axis=0)


In [126]:
#Gated CNN
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
for aspect in aspects :
    result = embeds[aspect]
    result = normalize(result, 'l2')
    scores = np.dot(result, result.T)
    scores[np.arange(len(val_idxs)), np.arange(len(val_idxs))] = -1000
    aucs = [0] * len(val_idxs)
    for i in range(len(val_idxs)) :
        aucs[i] = roc_auc_score(H[aspect][i], scores[i])
    print aspect, np.mean(aucs), np.argmax(aucs)

0 0.919458938987 2090
1 0.934243729845 4560
2 0.937620830739 380
3 0.965067441236 3893


In [37]:
def softmax(X, theta = 1.0, axis = None):
    """
    Compute the softmax of each element along an axis of X.

    Parameters
    ----------
    X: ND-Array. Probably should be floats. 
    theta (optional): float parameter, used as a multiplier
        prior to exponentiation. Default = 1.0
    axis (optional): axis to compute values along. Default is the 
        first non-singleton axis.

    Returns an array the same size as X. The result will sum to 1
    along the specified axis.
    """

    # make X at least 2d
    y = np.atleast_2d(X)

    # find axis
    if axis is None:
        axis = next(j[0] for j in enumerate(y.shape) if j[1] > 1)

    # multiply y against the theta parameter, 
    y = y * float(theta)

    # subtract the max for numerical stability
    y = y - np.expand_dims(np.max(y, axis = axis), axis)

    # exponentiate y
    y = np.exp(y)

    # take the sum along the specified axis
    ax_sum = np.expand_dims(np.sum(y, axis = axis), axis)

    # finally: divide elementwise
    p = y / ax_sum

    # flatten if X was 1D
    if len(X.shape) == 1: p = p.flatten()

    return p

In [57]:
clip_X = np.clip(X, 0, 1)

In [58]:
clip_X = clip_X.sum(axis=1)

In [59]:
len_X = np.argsort(clip_X)

In [70]:
clip_X[len_X[-1]]

305

In [72]:
clip_X[len_X[-1]]

305

In [46]:
t = "very dark beer . pours a nice finger and a half of creamy foam and stays throughout the beer . smells of coffee and roasted malt . has a major coffee taste with hints of chocolate . if you like black coffee , you will love this porter . creamy smooth mouthfeel and definitely gets smoother on the palate once it warms . it 's an ok porter but i feel there are much beer one 's out there ."

In [20]:
t1 = [vec.word2idx[x.text.lower()] if x.text.lower() in vec.word2idx else 1 for x in nlp(unicode(s))]# if x in vec.word2idx]
t2 = [0]*(305-len(t1)) + t1
X = np.vstack((X[:-1], t2))
print [x.text.lower() if x.text.lower() in vec.word2idx else 1 for x in nlp(unicode(s))]

[u'deep', u'amber', u'hue', u',', u'this', u'brew', u'is', u'topped', u'with', u'a', u'finger', u'of', u'off', u'white', u'head', u'.', u'smell', u'of', u'dog', 1, u',', u'green', 1, u',', u'and', u'slightly', u'fruity', u'.', u'taste', u'of', u'belgian', u'yeast', u',', u'coriander', u',', u'hard', u'water', u'and', u'bready', u'malt', u'.', u'light', u'body', u',', u'with', u'little', u'carbonation', u'.']


In [86]:
s = "Has a medium, white head, a hazy gold orange appearance, glass lacing, and some bubble action. Aroma is of a strong mango fruit whiff, with underlying white wheat bread. Taste is of those elements and a very mild, rounding hop bitterness. Mouthfeel is high medium, and Mango Wheat finishes midway between wet & dry. Great stuff, in the style. "

In [65]:
s= "Poured a dark brown color with a small, light brown head of foam. It smelled of chocolate, caramel and toastiness. Sweet chocolate flavor with a burnt toast maltiness. Same amazing smoothness as regular Dark Matter."

In [49]:
s = "A: The beer is jet black in color. It poured with a finger high dense tan head. The head died down, but consistently left a short head covering the surface. \
S: Light aromas of dark roasted malts and a bit of roast are present in the nose. \
T: The taste has flavors of dark and chocolate malts with a light amount of roast and hints of coffee beans. The roasted flavors become stronger in the finish and linger through the aftertaste. \
M: It feels medium-bodied and silky smooth on the palate with a moderate amount of carbonation."

In [107]:
s="his beer pours a deep black with medium sized head. The aroma features dark chocolate, licorice, brown sugar, and tobacco. The flavor is lovely and rich, with big brown sugar and roasty notes upfront. The mouthfeel is fairly full but not syrupy, with gentle carbonation."

In [19]:
s="Deep amber hue ,this brew is topped with a finger of off white head.\
Smell of dog hair, green olives, and slightly fruity.\
Taste of Belgian yeast, coriander, hard water and bready malt.\
Light body, with little carbonation."

In [9]:
s="Pours a dark brown-black with a finger of khaki foam. Head retention is alright with no lacing left behind. Aroma is burnt coffee, cigarettes, and toasted malts. Flavor profile is more smoky coffee, cigarettes, and toasted malts. Coffee is charred, burnt, and bitter. Mouth feel is medium-thin with a slightly watery, grainy texture. Overall, it comes off as smoked, bitter, and harsh. "

In [10]:
import spacy
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

In [151]:
[t.text.lower() for t in nlp(unicode(s))]

[u'deep',
 u'amber',
 u'hue',
 u',',
 u'well',
 u'filtered',
 u'and',
 u'only',
 u'a',
 u'little',
 u'carbonation',
 u',',
 u'this',
 u'brew',
 u'is',
 u'topped',
 u'with',
 u'a',
 u'finger',
 u'of',
 u'off',
 u'white',
 u'head',
 u'.',
 u'smell',
 u'of',
 u'caramel',
 u',',
 u'dog',
 u'hair',
 u',',
 u'green',
 u'olives',
 u',',
 u'and',
 u'slightly',
 u'fruity',
 u'.',
 u'taste',
 u'of',
 u'belgian',
 u'yeasts',
 u',',
 u'bubblegum',
 u',',
 u'coriander',
 u',',
 u'hard',
 u'water',
 u'and',
 u'bready',
 u'malt',
 u'.',
 u'light',
 u'body',
 u',',
 u'with',
 u'little',
 u'carbonation',
 u'.']

In [161]:
vec.word2idx['olives']

KeyError: 'olives'